In [10]:
import os
from dotenv import load_dotenv
import boto3
from botocore.client import Config
from botocore.exceptions import ClientError

load_dotenv()

# Get environment variables with error handling
SPACES_ENDPOINT = os.getenv('SPACES_ENDPOINT')
SPACES_KEY = os.getenv('SPACES_KEY')
SPACES_SECRET = os.getenv('SPACES_SECRET')
SPACE_NAME = os.getenv('SPACE_NAME')
SPACE_REGION = os.getenv("SPACE_REGION", "sgp1")

required_vars = ["SPACES_ENDPOINT", "SPACES_KEY", "SPACES_SECRET", "SPACE_NAME"]
missing_vars = [var for var in required_vars if os.getenv(var) is None]

if missing_vars:
    raise ValueError(f"Missing required environment variables: {', '.join(missing_vars)}")

try:
    session = boto3.session.Session()
    client = session.client('s3',
                            endpoint_url=SPACES_ENDPOINT,
                            config=Config(s3={'addressing_style': 'virtual'}),
                            region_name=SPACE_REGION,
                            aws_access_key_id=SPACES_KEY,
                            aws_secret_access_key=SPACES_SECRET)

    # --- Upload Object ---
    file_content = b'Hello, World!\nThis is a test from the complete code.'
    object_key = 'heollo-folder/hello-world.txt' # Change this if you want a different path/filename

    try:
        upload_response = client.put_object(Bucket=SPACE_NAME,
                                     Key=object_key,
                                     Body=file_content,
                                     ACL='private',  # Or 'public-read'
                                     Metadata={'x-amz-meta-my-key': 'your-value'})

        if upload_response['ResponseMetadata']['HTTPStatusCode'] == 200:
            print(f"File '{object_key}' uploaded successfully to Space '{SPACE_NAME}'.")
        else:
            print(f"File upload failed. Response: {upload_response}")
    except ClientError as e:
        print(f"Upload Error: {e}")

    # --- List Objects ---
    try:
        list_response = client.list_objects_v2(Bucket=SPACE_NAME)
        if 'Contents' in list_response:
            print("\nObjects in the bucket:")
            for obj in list_response['Contents']:
                print(f"  - {obj['Key']}")
        else:
            print("\nBucket is empty.")
    except ClientError as e:
        print(f"List Error: {e}")


except Exception as e:
    print(f"An unexpected error occurred: {e}")

An unexpected error occurred: SSL validation failed for https://jef-general-bucket.jef-general-bucket.sgp1.digitaloceanspaces.com/heollo-folder/hello-world.txt hostname 'jef-general-bucket.jef-general-bucket.sgp1.digitaloceanspaces.com' doesn't match either of '*.sgp1.digitaloceanspaces.com', 'sgp1.digitaloceanspaces.com'
